# Pandas Introduction Part 4

## Overview

This notebook uses the IMDB dataset from Kaggle:  
https://www.kaggle.com/PromptCloudHQ/imdb-data#IMDB-Movie-Data.csv

Three different methods for quering data that is not "tidy" are presented.  Two of these reformat the data to be tidy in different ways.  A third makes use of an auxiliary mapping maintained in a Python dictionary.

The same queries are formulated using all three methods.

In [1]:
import pandas as pd
import numpy as np

## Read Movie Data Set
This data was read in and preprocessed in Pandas02.  Load that notebook.

In [2]:
import pickle
with open('../data/movies.pickle','rb') as p:
    movies = pickle.load(p)

In [3]:
# display first 2 rows of the DataFrame
movies.head(2)

,ID,Genre,Description,Director,Actors,Year,Runtime,Rating,Votes,Revenue,Metascore
Title,,,,,,,,,,,
Guardians of the Galaxy,1,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
Prometheus,2,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0


## Data Wrangling: Queries involving Genre
Each cell in the Genre column is a single string.  This string consists of several genres separated by commas.  The Actors column is the same.

With data in this format, it is not obvious how to answer queries such as "what genre has the highest average rating?". Or "what actor has the highest average rating?".

There are (at least) 3 ways to wrangle this column's data:
1. Create an indicator column for the presence/absence of each genre for each movie.
2. Create a mapping of genre to titles and a create a custom "groupby" to yield groups of movies for each genre.  The normal Pandas groupby won't work appropriately for genre or actors.
3. "Denormalize" the data.

In all cases, it is helpful to begin by creating a list of unique Genres (or unique Actors) before doing the data analysis.

Briefly, the pros and cons of the above are:
1. This may be best if a machine learning algorithm is to be used on the data.  It also makes data analysis easy and it is easy to set up.
2. This solution only takes a couple of lines of code, however it may be difficult for a junior data analyst to understand and maintain. If offers good speed and good memory usage.
3. This is the easiest form for a data analyst to work with, and is relatively easy to set up.  However it may require significant additional memory.

Examples of each of the above are shown below.

### Quick Look at the Data

In [ ]:
# Look at Genre as is
movies['Genre'].head(5)

In [ ]:
# typical way of finding unique values
# works when cell contains one value
movies['Director'].value_counts().head()

In [ ]:
# this doesnt work as needed for Genre or Actor
movies['Genre'].value_counts().head()

The above is count per genre string, not per genre.

### Write Parser for Genre (or Actors) String

In [ ]:
def parse_string(s):
    """Convert string with comma separated values to list of strings.
    For each string, remove leading and trailing whitespace.
    """
    values = s.split(',')
    return [value.strip() for value in values]

#### Use Parser
1. use Pandas Vectorized operator 'apply' to avoid explict iteration over all titles
2. note: index of result Series is Title
3. note: value of result Series is list of genres

The same mapping could have been created using a defaultdict(list) and explicitly iterating over each movie, but Pandas makes this easier and possibly faster.

In [ ]:
# create a Series with values being the parsed genre string
title_genre_series = movies['Genre'].apply(parse_string)
title_genre_series.head()

For each title, add the list of genres to the genre set.

In [ ]:
# find the unique genres
genres = set()
title_genre_series.apply(lambda x: genres.update(x))
genres

## Create Indicator Variables -- Method 1
It is helpful to prefix the name of each new column so they can be analyzed together.

Each indicator column will have True for presence of genre and False for absence of genre.

In [ ]:
for genre in genres:
    movies['Genre_'+genre] = movies['Genre'].apply(lambda x: genre in x)

In [ ]:
# select all columns that begin with Genre_
genre_indicators = movies.filter(regex='^Genre_', axis='columns')
genre_indicators.head(2)

In [ ]:
# Pandas uses bool for datatype for each column, so memory usage is efficient
genre_indicators.dtypes.value_counts()

### Compute Movies per Genre -- Indicator Variables

In [ ]:
# one line of code to compute how many movies of each genre
movies_per_genre = genre_indicators.sum(axis='rows').to_dict()
movies_per_genre

### Find Highest Rated Movies: Action & Sci-Fi -- Indicator Variables

In [ ]:
criteria = movies['Genre_Action'] & movies['Genre_Sci-Fi']
movies_action_sci_fi = movies[criteria]
movies_action_sci_fi.nlargest(3, 'Rating')

### Find Directors with the most Different Genre

In [ ]:
for director, group_df in movies.groupby('Director'):
    print(director)
    indicator_cols = group_df.filter(regex='^Genre_', axis = 'columns')
    num_genres = indicator_cols.sum(axis=0)
    print(num_genres)

In [ ]:
# genre column list
genre_cols = movies.filter(regex='^Genre_', axis=1).columns
genre_cols

In [ ]:
g = movies.groupby('Director')
g.get_group('Christopher Nolan')[genre_cols].any().sum()

## Create Mapping of Genre to Titles -- Method 2

In [ ]:
# assuming indicator columns do not exist
genre_to_titles = {}
for genre in genres:
    genre_boolean_series = movies['Genre'].apply(lambda x: genre in x)
    titles = movies[genre_boolean_series].index.tolist()
    genre_to_titles[genre] = titles

### Compute Movies per Genre -- Genre-to-Titles Mapping

In [ ]:
# one line dictionary comprehension
movies_per_genre2 = {'Genre_'+genre:len(titles) for genre,titles in genre_to_titles.items()}
movies_per_genre2

In [ ]:
# verify results are the same
movies_per_genre == movies_per_genre2

### Find Highest Rated Movies: Action & Sci-Fi -- Genre-to-Titles Mapping

In [ ]:
action_titles = set(genre_to_titles['Action'])
sci_fi_titles = set(genre_to_titles['Sci-Fi'])
titles = action_titles.intersection(sci_fi_titles)
movies_action_sci_fi2 = movies.loc[titles]
movies_action_sci_fi2.nlargest(3, 'Rating')

In [ ]:
# verify they are the same
# need to sort them before comparison
df_1 = movies_action_sci_fi.sort_index()
df_2 = movies_action_sci_fi2.sort_index()
df_1.equals(df_2)

### Define Custom (Overlapping) Groups
The genre to titles mapping is a one-to-many mapping.

Generators are memory efficient.  The dataframe per value is created only as necessary and if there is no reference to it, Python's garbage collector can reclaim the memory.

In [ ]:
def custom_groupby(df, mapping, values):
    for value in values:
        yield (value, df.loc[mapping[value]])

In [ ]:
groups = custom_groupby(movies, genre_to_titles, genres)
for name, group_df in groups:
    print(f"{name:<9} Rating: {group_df['Rating'].mean():4.2f} Count: {len(group_df):>3}")

In [ ]:
# same as above, but store result in df
rows = []
groups = custom_groupby(movies, genre_to_titles, genres)
for name, group_df in groups:
    row = (name, group_df['Rating'].mean(), len(group_df))
    rows.append(row)
df = pd.DataFrame(data=rows, columns=['Genre', 'Rating', 'Count'])
df.set_index('Genre', inplace=True)
df

In [ ]:
# display the 5 highest avg ratings by genre in Ratings order
df.nlargest(n=5, columns='Rating')

In [ ]:
# display the 5 highest avg ratings by genre having at least 20 movies
df_20 = df[df['Count'] >= 20]
df_20.nlargest(5, 'Rating')

## Denormalize -- Method 3

In [ ]:
# create the one-to-many "relation" between genre and titles
rows = []
for key in genre_to_titles:
    for value in genre_to_titles[key]:
        rows.append((key, value))
df_genre_title = pd.DataFrame(data=rows, columns=['Genre', 'Title'])
df_genre_title.set_index('Title', inplace=True)
df_genre_title.head()

In [ ]:
# join movies with the one-to-many table
df_full = df_genre_title.join(movies, rsuffix='_right')
df_full.drop('Genre_right', axis=1, inplace=True)
df_full.head(2)

In [ ]:
# create a multi-index on Genre, Title
# df_full.reset_index(inplace=True)
# df_full.set_index(['Genre', 'Title'], inplace=True)
# df_full.sort_index(inplace=True)

In [ ]:
df_full.sort_values('Genre', inplace=True)

In [ ]:
# this "denormalization" is using 2.5 times as much space
len(df_full), len(movies)

### Compute Movies per Genres -- Denormalized
df.groupby will now work for each genre

In [ ]:
for name, group_df in df_full.groupby('Genre'):
    print(f'{name:<10} {len(group_df):>3}')

### Find Highest Rated Movies: Action & Sci-Fi -- Denormalize

In [ ]:
g = df_full.groupby('Genre')
action = g.get_group('Action')
sci_fi = g.get_group('Sci-Fi')
common = action.index.intersection(sci_fi.index)

In [ ]:
movies.loc[common].nlargest(3, 'Rating')

In [ ]:
# now groupby will work for each Genre
g = df_full.groupby('Genre')
g['Rating'].mean()

In [ ]:
# compute count per genre using the genre_to_titles dictionary
genres_to_count = {genre:len(titles) for genre,titles in genre_to_titles.items()}
genres_to_count

In [ ]:
g = movies.groupby(by=genre_to_titles)
for name, group in g:
    print(name)
    print(group['Genre'])

In [ ]:
# find highest rated action movie on the fly
action_titles = movies['Genre'].apply(lambda x: 'Action' in x)
action_movies = movies.loc[action_titles]
action_movies[action_movies['Rating'] == action_movies['Rating'].max()]

In [ ]:
# find highest rated action movie using Indicator Variable: Genre_Action
action_movies = movies[movies['Genre_Action']]
action_movies[action_movies['Rating'] == action_movies['Rating'].max()]

In [ ]:
# find the highest rated action movie using dictionary
action_titles = genre_to_titles['Action']
action_movies = movies.loc[action_titles]
action_movies[action_movies['Rating'] == action_movies['Rating'].max()]

In [ ]:
%%timeit
# find highest rated action movie on the fly
action_titles = movies['Genre'].apply(lambda x: 'Action' in x)
action_movies = movies[action_titles]
action_movies[action_movies['Rating'] == action_movies['Rating'].max()]

In [ ]:
%%timeit
# find highest rated action movie using precomputed Indicator Variable: Genre_Action
action_movies = movies[movies['Genre_Action']]
action_movies[action_movies['Rating'] == action_movies['Rating'].max()]

In [ ]:
%%timeit
# find the highest rated action movie using precomputed dictionary
action_titles = genre_to_titles['Action']
action_movies = movies.loc[action_titles]
action_movies[action_movies['Rating'] == action_movies['Rating'].max()]

In [ ]:
# find Sci-Fi movies
sci_fi_movies = movies[movies['Genre_Sci-Fi']]
sci_fi_movies.head(2)

In [ ]:
# find movies that are both action and sci-fi
action_sci_fi_movies = movies[movies['Genre_Action'] & movies['Genre_Sci-Fi']]
action_sci_fi_movies.head(3)

In [ ]:
# find highest rated movie that is both Action and Sci-Fi
movie_subset = movies[movies['Genre_Action'] & movies['Genre_Sci-Fi']]
movie_subset[movie_subset['Rating'] == movie_subset['Rating'].max()]

In [ ]:
# find the avg rating of all action movies
action_movies = movies[movies['Genre_Action']]
action_movies['Rating'].mean()

In [ ]:
# usually groupby is not used on a boolean series, but it works
# the group of interest is the True group
groupby = movies.groupby('Genre_Action')

for name, group in groupby:
    print(name, np.round(group['Rating'].mean(),4), group['Rating'].count())

### Group By
Find the highest rated directors who directed at least 5 movies

In [ ]:
def group_filter(g):
    if g['Rating'].count() >= 5:
        return g['Rating'].mean()

In [ ]:
s = movies.groupby('Director').apply(group_filter)
s[s.notnull()].nlargest(5)

## Data Wrangling: Queries involving Actor
As there are so many different actors, it's probably better to rely on dictionaries instead of creating a great many indicator columns.

In [ ]:
def parse_genre_string(s):
    """Convert string with comma separated values to list of strings.
    For each string, remove leading and trailing whitespace.
    """
    genres = s.split(',')
    return [genre.strip() for genre in genres]

In [ ]:
# parsing of Genre is identical to parsing of Actors
movies['Actors'].apply(parse_genre_string).head(5)

In [ ]:
# find the set of unique actors
actors = set()
title_actors_series = movies['Actors'].apply(parse_genre_string)
title_actors_series.apply(lambda x: actors.update(x))
len(actors)

#### Using an Inverted Dictionary to Compute Movies per Actor

In [ ]:
# create a mapping from genre to titles
from collections import defaultdict
actor_to_titles = defaultdict(list)
for actor in actors:
    titles = movies['Actors'].apply(lambda x: actor in x)
    titles = movies[titles].index.tolist()
    for title in titles:
        actor_to_titles[actor].append(title)

In [ ]:
# compute count per genre using the genre_to_titles dictionary
actors_to_count = {actor:len(titles) for actor,titles in actor_to_titles.items()}
len(actors_to_count)

In [ ]:
# try inverted dictionary
movies.loc[actor_to_titles['Zooey Deschanel']]

In [ ]:
# find the average rating of actors in at least 5 movies
actor_rating = {}
for actor in actors:
    movies_with_actor = movies.loc[actor_to_titles[actor]]
    if movies_with_actor['Rating'].count() >= 5:
        actor_rating[actor] = movies_with_actor['Rating'].mean()

In [ ]:
# read in IMDB move dataset
movies = pd.read_csv('../data/IMDB-Movie-Data.csv', index_col='Title')
movies.head(2)

# HERE HERE
A dictionary is more memory efficient than denormalizing the table, but denormalizing is much easier to work with!

In [ ]:
def custom_groupby(df, mapping, values):
    for value in values:
        yield (value, df.loc[mapping[value]])

In [ ]:
movies = movies_copy.copy()
movies = movies.reset_index(drop=False).set_index('Title', drop=True)

In [ ]:
movies.head(1)

In [ ]:
np.std([1, 2, 3])

In [ ]:
df = pd.DataFrame([1, 2, 3], columns=['A'])
df.apply(np.std)

In [ ]:
df.columns

In [ ]:
my_group = custom_groupby(movies, actor_to_titles, ['Zooey Deschanel', 'John Goodman'])

In [ ]:
from IPython.display import display
for name, group in my_group:
    print(name, len(group), group['Rating'].mean(), group['Metascore'].std())
    print()
    display(group)

In [ ]:
def gen1(n):
    a = 0
    b = 1
    for i in range(n):
        a, b = a+b, a
        yield(a)

In [ ]:
g = gen1(7)

In [ ]:
tuples = []
for actor in actors:
    for title in actor_to_titles[actor]:
        tuples.append((actor, title))
        
df_actor = pd.DataFrame(data=tuples, columns=['Actor','Title'])
df_actor.set_index('Actor', drop=True, inplace=True)

In [ ]:
df_actor.head()

In [ ]:
df = df_actor.join(movies, on='Title', how='left')
df.head()

In [ ]:
len(df_actor)

In [ ]:
len(movies.index)

In [ ]:
len(set(movies.index))

In [ ]:
m2 = movies.reset_index()
b = m2.duplicated(subset='Title', keep='first')
m2[b]

In [ ]:
b = m2.duplicated(subset='Title', keep='last')
m2[b]

In [ ]:
df2 = df.set_index(index)

In [ ]:
actor_to_titles

In [ ]:
df_actor.head()

In [ ]:
tuples = []
for actor in actors:
    for title in actor_to_titles[actor]:
        tuples.append((actor, title))
        
index = pd.MultiIndex.from_tuples(tuples, names=['Actor', 'Title'])        

In [ ]:
movies.set_index(index)

In [ ]:
# its easier to work with dataframes than dictionaries
df = pd.DataFrame(data=[], columns=['Rating', 'Count'])
dfs = []
for actor in actors:
    df = movies.loc[actor_to_titles[actor]]
    df['Actor'] = actor
    df.set_index('Actor', drop=False, inplace=True)
    dfs.append(df)
df_new = pd.concat(dfs)
df_new.head()

In [ ]:
df_actor_rating = pd.DataFrame.from_dict(actor_rating, orient='index')
df_actor_rating.rename_axis('Actor')
df_actor_rating.columns = ['Rating']
df_actor_rating.nlargest(n=5, columns='Rating')

In [ ]:
df_actor_rating.columns

In [ ]:
df_actor_rating.index

In [ ]:
# sort by value
actor_rating_sorted = sorted(actor_rating.items(), key=lambda x: x[1], reverse=True)
for actor, rating in actor_rating_sorted:
    print(f'{actor}: {rating:4.2f}')

In [ ]:
# get the average rating and number of movies for each actor in at least 5 movies
actor_rating = {}
for actor in actors_to_title:
    titles = actors_to_title[actor]
    movies_with_actor = movie.loc[titles]
    count = movies_with_actor['Rating'].count()
    if count >= 5:
        actor_rating[actor] = movies_with_actor['Rating'].mean()

In [ ]:
actor_rating_sorted = sorted(actor_rating.items(), key=lambda kv: kv[1], reverse=True)
actor_rating_sorted[:10]

In [ ]:
movie.loc[actor_to_titles['Leonardo DiCaprio']]

In [ ]:
# with dictionaries, its slighter harder to query for two actors, than with indicator columns
# find all movies with Leonardo DiCaprio and Matt Damon
titles1 = actors_to_title['Leonardo DiCaprio']
titles2 = actors_to_title['Matt Damon']
titles = set(titles1).intersection(set(titles2))
titles
movie.loc[titles]